# 07 - Visualization & Report Generation

Publication-ready figures and tables for "Categories of the Commons" research.

## Outputs
- Figure 1: OSS Governance Taxonomy Overview
- Figure 2: Entropy Distribution by Category
- Figure 3: VSM Radar Comparison
- Figure 4: Category Theory Composition Diagram
- Table 1: Project Classification Summary
- Table 2: Statistical Metrics by Category

In [ ]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.gridspec import GridSpec
import seaborn as sns
from pathlib import Path
from typing import Dict, List, Any, Tuple
import warnings
warnings.filterwarnings('ignore')

# Publication-quality settings
plt.rcParams.update({
    'font.size': 11,
    'font.family': 'serif',
    'axes.labelsize': 12,
    'axes.titlesize': 13,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'figure.titlesize': 14,
    'figure.dpi': 150,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight',
    'savefig.pad_inches': 0.1
})

# Color palette for governance categories
CATEGORY_COLORS = {
    'Stadium': '#E74C3C',      # Red - high visibility, concentrated
    'Federation': '#3498DB',   # Blue - distributed, collaborative
    'Club': '#2ECC71',         # Green - community-focused
    'Toy': '#9B59B6',          # Purple - experimental
    'Hybrid': '#F39C12'        # Orange - mixed characteristics
}

# Create output directories
FIGURES_DIR = Path('../outputs/figures')
TABLES_DIR = Path('../outputs/tables')
FIGURES_DIR.mkdir(parents=True, exist_ok=True)
TABLES_DIR.mkdir(parents=True, exist_ok=True)

print("Visualization environment configured for publication output")

## Load Project Data

In [ ]:
import sys
sys.path.insert(0, '../src')
from analysis.entropy_calculation import EntropyCalculator

def load_all_project_data() -> Dict[str, Dict]:
    """Load all collected project data."""
    data_dir = Path('../data/raw')
    projects = {}
    
    for json_file in data_dir.glob('*_data.json'):
        try:
            with open(json_file, 'r') as f:
                data = json.load(f)
                project_name = data.get('full_name', json_file.stem.replace('_data', ''))
                projects[project_name] = data
        except Exception as e:
            print(f"Error loading {json_file}: {e}")
    
    print(f"Loaded {len(projects)} projects")
    return projects

def classify_all_projects(projects: Dict[str, Dict]) -> pd.DataFrame:
    """Classify all projects and create summary DataFrame."""
    calculator = EntropyCalculator()
    results = []
    
    for name, data in projects.items():
        contributors = data.get('contributors', [])
        classification = calculator.classify_project(contributors)
        
        # Determine primary category
        class_label = classification['classification']
        if 'Stadium' in class_label:
            category = 'Stadium'
        elif 'Federation' in class_label or 'Club' in class_label:
            category = 'Federation'
        elif 'Hybrid' in class_label or 'Uncertain' in class_label:
            category = 'Hybrid'
        else:
            category = 'Unknown'
        
        metrics = classification.get('metrics', {})
        results.append({
            'project': name,
            'classification': class_label,
            'category': category,
            'confidence': classification.get('confidence', 0),
            'stadium_score': classification.get('stadium_score', 0),
            'entropy': metrics.get('entropy', 0),
            'normalized_entropy': metrics.get('normalized_entropy', 0),
            'gini': metrics.get('gini_coefficient', 0),
            'top1_pct': metrics.get('top_contributor_pct', 0),
            'contributors': metrics.get('total_contributors', 0),
            'stars': data.get('stargazers_count', 0),
            'forks': data.get('forks_count', 0),
            'open_issues': data.get('open_issues_count', 0)
        })
    
    return pd.DataFrame(results)

# Load and classify
projects = load_all_project_data()
df = classify_all_projects(projects)
df.sort_values('stars', ascending=False)

## Figure 1: OSS Governance Taxonomy Overview

A comprehensive visualization showing the relationship between Asparouhova's taxonomy,
entropy measures, and the projects in our dataset.

In [ ]:
def create_taxonomy_figure(df: pd.DataFrame) -> plt.Figure:
    """Create taxonomy overview figure with quadrant plot."""
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Left: Entropy vs Gini scatter with quadrants
    ax1 = axes[0]
    
    # Draw quadrant backgrounds
    ax1.axhline(y=0.8, color='gray', linestyle='--', alpha=0.5)
    ax1.axvline(x=0.6, color='gray', linestyle='--', alpha=0.5)
    
    # Add quadrant labels
    ax1.text(0.3, 0.9, 'STADIUM\n(High inequality,\nlow diversity)', 
             ha='center', va='center', fontsize=9, alpha=0.7,
             bbox=dict(boxstyle='round', facecolor='#E74C3C', alpha=0.2))
    ax1.text(0.8, 0.9, 'HYBRID\n(High inequality,\nhigh diversity)', 
             ha='center', va='center', fontsize=9, alpha=0.7,
             bbox=dict(boxstyle='round', facecolor='#F39C12', alpha=0.2))
    ax1.text(0.3, 0.6, 'CLUB\n(Low inequality,\nlow diversity)', 
             ha='center', va='center', fontsize=9, alpha=0.7,
             bbox=dict(boxstyle='round', facecolor='#2ECC71', alpha=0.2))
    ax1.text(0.8, 0.6, 'FEDERATION\n(Low inequality,\nhigh diversity)', 
             ha='center', va='center', fontsize=9, alpha=0.7,
             bbox=dict(boxstyle='round', facecolor='#3498DB', alpha=0.2))
    
    # Plot projects
    for category in df['category'].unique():
        subset = df[df['category'] == category]
        color = CATEGORY_COLORS.get(category, 'gray')
        ax1.scatter(subset['normalized_entropy'], subset['gini'],
                   c=color, label=category, s=100, alpha=0.7, edgecolors='white')
        
        # Add project labels
        for _, row in subset.iterrows():
            short_name = row['project'].split('/')[-1][:10]
            ax1.annotate(short_name, (row['normalized_entropy'], row['gini']),
                        xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    ax1.set_xlabel('Normalized Entropy (diversity)')
    ax1.set_ylabel('Gini Coefficient (inequality)')
    ax1.set_title('(a) Governance Classification Space')
    ax1.set_xlim(0, 1)
    ax1.set_ylim(0.5, 1)
    ax1.legend(loc='lower right')
    ax1.grid(True, alpha=0.3)
    
    # Right: Category distribution bar chart
    ax2 = axes[1]
    category_counts = df['category'].value_counts()
    colors = [CATEGORY_COLORS.get(cat, 'gray') for cat in category_counts.index]
    bars = ax2.bar(category_counts.index, category_counts.values, color=colors, alpha=0.8)
    
    ax2.set_xlabel('Governance Category')
    ax2.set_ylabel('Number of Projects')
    ax2.set_title('(b) Distribution of Classifications')
    
    # Add value labels on bars
    for bar, count in zip(bars, category_counts.values):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                str(count), ha='center', va='bottom', fontsize=11)
    
    plt.tight_layout()
    return fig

if len(df) > 0:
    fig1 = create_taxonomy_figure(df)
    fig1.savefig(FIGURES_DIR / 'fig1_taxonomy_overview.png')
    fig1.savefig(FIGURES_DIR / 'fig1_taxonomy_overview.pdf')
    print(f"Saved Figure 1 to {FIGURES_DIR}")
    plt.show()
else:
    print("No project data available for visualization")

## Figure 2: Entropy Distribution Analysis

In [ ]:
def create_entropy_figure(df: pd.DataFrame, projects: Dict) -> plt.Figure:
    """Create entropy distribution visualization."""
    fig = plt.figure(figsize=(14, 10))
    gs = GridSpec(2, 2, figure=fig, hspace=0.3, wspace=0.3)
    
    # (a) Entropy histogram by category
    ax1 = fig.add_subplot(gs[0, 0])
    for category in df['category'].unique():
        subset = df[df['category'] == category]['entropy']
        if len(subset) > 0:
            ax1.hist(subset, bins=10, alpha=0.6, label=category,
                    color=CATEGORY_COLORS.get(category, 'gray'))
    ax1.set_xlabel('Shannon Entropy (bits)')
    ax1.set_ylabel('Frequency')
    ax1.set_title('(a) Entropy Distribution by Category')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # (b) Box plot comparison
    ax2 = fig.add_subplot(gs[0, 1])
    categories = df['category'].unique()
    box_data = [df[df['category'] == cat]['normalized_entropy'].dropna() for cat in categories]
    bp = ax2.boxplot(box_data, labels=categories, patch_artist=True)
    for patch, cat in zip(bp['boxes'], categories):
        patch.set_facecolor(CATEGORY_COLORS.get(cat, 'gray'))
        patch.set_alpha(0.6)
    ax2.set_ylabel('Normalized Entropy')
    ax2.set_title('(b) Entropy Spread by Category')
    ax2.grid(True, alpha=0.3, axis='y')
    
    # (c) Top contributor % vs entropy
    ax3 = fig.add_subplot(gs[1, 0])
    for category in df['category'].unique():
        subset = df[df['category'] == category]
        ax3.scatter(subset['top1_pct'], subset['entropy'],
                   c=CATEGORY_COLORS.get(category, 'gray'),
                   label=category, s=80, alpha=0.7)
    ax3.set_xlabel('Top Contributor %')
    ax3.set_ylabel('Shannon Entropy (bits)')
    ax3.set_title('(c) Dominance vs Entropy')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # (d) Contributor Lorenz curves for select projects
    ax4 = fig.add_subplot(gs[1, 1])
    
    # Plot perfect equality line
    ax4.plot([0, 1], [0, 1], 'k--', label='Perfect Equality', alpha=0.5)
    
    # Plot Lorenz curves for up to 5 projects
    for project_name in list(projects.keys())[:5]:
        data = projects[project_name]
        contributors = data.get('contributors', [])
        if not contributors:
            continue
            
        contributions = sorted([c.get('contributions', 0) for c in contributors], reverse=True)
        total = sum(contributions)
        if total == 0:
            continue
            
        # Calculate cumulative percentages
        cumulative = np.cumsum(contributions) / total
        x = np.arange(1, len(cumulative) + 1) / len(cumulative)
        
        short_name = project_name.split('/')[-1]
        ax4.plot(x, cumulative, label=short_name, linewidth=2, alpha=0.7)
    
    ax4.set_xlabel('Cumulative % of Contributors')
    ax4.set_ylabel('Cumulative % of Contributions')
    ax4.set_title('(d) Lorenz Curves (Contribution Inequality)')
    ax4.legend(loc='lower right', fontsize=8)
    ax4.grid(True, alpha=0.3)
    ax4.set_xlim(0, 1)
    ax4.set_ylim(0, 1)
    
    return fig

if len(df) > 0:
    fig2 = create_entropy_figure(df, projects)
    fig2.savefig(FIGURES_DIR / 'fig2_entropy_analysis.png')
    fig2.savefig(FIGURES_DIR / 'fig2_entropy_analysis.pdf')
    print(f"Saved Figure 2 to {FIGURES_DIR}")
    plt.show()
else:
    print("No project data available")

## Figure 3: VSM Radar Comparison

Comparing projects across Viable System Model dimensions (S1-S5).

In [ ]:
def calculate_vsm_metrics(project_data: Dict) -> Dict[str, float]:
    """Calculate VSM S1-S5 metrics for a project."""
    contributors = project_data.get('contributors', [])
    
    # S1: Operations (commit activity)
    recent_commits = len(project_data.get('recent_commits', []))
    s1 = min(recent_commits / 100, 1.0)  # Normalize to 0-1
    
    # S2: Coordination (issues/PR management)
    open_issues = project_data.get('open_issues_count', 0)
    closed_issues = project_data.get('closed_issues_count', 0)
    total_issues = open_issues + closed_issues
    s2 = closed_issues / total_issues if total_issues > 0 else 0.5
    
    # S3: Control (contributor distribution)
    if contributors:
        contributions = [c.get('contributions', 0) for c in contributors]
        total = sum(contributions)
        top_pct = contributions[0] / total if total > 0 else 0
        s3 = 1 - top_pct  # Lower concentration = better control distribution
    else:
        s3 = 0.5
    
    # S4: Intelligence (documentation, community)
    has_wiki = 1.0 if project_data.get('has_wiki', False) else 0.0
    has_discussions = 1.0 if project_data.get('has_discussions', False) else 0.0
    s4 = (has_wiki + has_discussions) / 2
    
    # S5: Identity (stars, forks as community recognition)
    stars = project_data.get('stargazers_count', 0)
    forks = project_data.get('forks_count', 0)
    s5 = min((stars + forks * 2) / 100000, 1.0)  # Normalize
    
    return {'S1': s1, 'S2': s2, 'S3': s3, 'S4': s4, 'S5': s5}

def create_vsm_radar(projects: Dict, max_projects: int = 6) -> plt.Figure:
    """Create VSM radar chart comparing projects."""
    categories = ['S1\nOperations', 'S2\nCoordination', 'S3\nControl', 
                  'S4\nIntelligence', 'S5\nIdentity']
    N = len(categories)
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]  # Complete the loop
    
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
    
    project_names = list(projects.keys())[:max_projects]
    colors = plt.cm.Set2(np.linspace(0, 1, len(project_names)))
    
    for idx, project_name in enumerate(project_names):
        vsm = calculate_vsm_metrics(projects[project_name])
        values = [vsm['S1'], vsm['S2'], vsm['S3'], vsm['S4'], vsm['S5']]
        values += values[:1]  # Complete the loop
        
        short_name = project_name.split('/')[-1]
        ax.plot(angles, values, 'o-', linewidth=2, label=short_name, color=colors[idx])
        ax.fill(angles, values, alpha=0.15, color=colors[idx])
    
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories, size=11)
    ax.set_ylim(0, 1)
    ax.set_title('VSM Subsystem Comparison', size=14, y=1.08)
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
    ax.grid(True)
    
    return fig

if len(projects) > 0:
    fig3 = create_vsm_radar(projects)
    fig3.savefig(FIGURES_DIR / 'fig3_vsm_radar.png')
    fig3.savefig(FIGURES_DIR / 'fig3_vsm_radar.pdf')
    print(f"Saved Figure 3 to {FIGURES_DIR}")
    plt.show()
else:
    print("No project data available")

## Figure 4: Theoretical Framework Diagram

Illustrating the category-theoretic composition of governance models.

In [ ]:
def create_framework_diagram() -> plt.Figure:
    """Create theoretical framework composition diagram."""
    fig, ax = plt.subplots(figsize=(14, 10))
    ax.set_xlim(0, 14)
    ax.set_ylim(0, 10)
    ax.axis('off')
    
    # Define boxes for each theoretical component
    boxes = {
        'GitHub Data': (1, 7.5, 2.5, 1.5, '#E8F4F8'),
        'Shannon Entropy': (5, 7.5, 2.5, 1.5, '#FFF3E0'),
        'VSM S1-S5': (9, 7.5, 2.5, 1.5, '#E8F5E9'),
        'Asparouhova\nTaxonomy': (1, 4, 2.5, 1.5, '#FCE4EC'),
        'Ostrom\nPrinciples': (5, 4, 2.5, 1.5, '#F3E5F5'),
        'Category\nTheory': (9, 4, 2.5, 1.5, '#E3F2FD'),
        'Governance\nClassification': (5, 0.5, 3, 2, '#FFEBEE')
    }
    
    # Draw boxes
    for name, (x, y, w, h, color) in boxes.items():
        rect = mpatches.FancyBboxPatch((x, y), w, h,
                                        boxstyle="round,pad=0.05",
                                        facecolor=color,
                                        edgecolor='black',
                                        linewidth=2)
        ax.add_patch(rect)
        ax.text(x + w/2, y + h/2, name, ha='center', va='center',
               fontsize=11, fontweight='bold')
    
    # Draw arrows with labels
    arrow_style = dict(arrowstyle='->', color='#333', lw=2,
                       connectionstyle='arc3,rad=0')
    
    # Vertical arrows (data flow)
    arrows = [
        ((2.25, 7.5), (2.25, 5.5), 'collect'),
        ((6.25, 7.5), (6.25, 5.5), 'calculate'),
        ((10.25, 7.5), (10.25, 5.5), 'map'),
        ((2.25, 4), (5, 2), 'categorize'),
        ((6.25, 4), (6.5, 2.5), 'validate'),
        ((10.25, 4), (8, 2), 'formalize'),
    ]
    
    for (x1, y1), (x2, y2), label in arrows:
        ax.annotate('', xy=(x2, y2), xytext=(x1, y1),
                   arrowprops=arrow_style)
        mid_x, mid_y = (x1 + x2) / 2, (y1 + y2) / 2
        ax.text(mid_x + 0.3, mid_y, label, fontsize=9, style='italic', alpha=0.8)
    
    # Add title and labels
    ax.text(7, 9.5, 'Categories of the Commons: Theoretical Framework',
           ha='center', fontsize=16, fontweight='bold')
    
    # Add layer labels
    ax.text(0.3, 8.25, 'Data Layer', fontsize=10, rotation=90, va='center')
    ax.text(0.3, 4.75, 'Theory Layer', fontsize=10, rotation=90, va='center')
    ax.text(0.3, 1.5, 'Output Layer', fontsize=10, rotation=90, va='center')
    
    # Add functor notation
    ax.text(12.5, 4.75, 'F: OSS → Gov', fontsize=10, style='italic',
           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    ax.text(12.5, 4.25, 'Functor mapping', fontsize=8, alpha=0.7)
    
    return fig

fig4 = create_framework_diagram()
fig4.savefig(FIGURES_DIR / 'fig4_framework_diagram.png')
fig4.savefig(FIGURES_DIR / 'fig4_framework_diagram.pdf')
print(f"Saved Figure 4 to {FIGURES_DIR}")
plt.show()

## Table 1: Project Classification Summary

In [ ]:
def create_classification_table(df: pd.DataFrame) -> pd.DataFrame:
    """Create publication-ready classification table."""
    table_df = df[['project', 'classification', 'confidence', 'entropy', 
                   'gini', 'top1_pct', 'contributors', 'stars']].copy()
    
    # Format columns
    table_df['confidence'] = (table_df['confidence'] * 100).round(0).astype(int).astype(str) + '%'
    table_df['entropy'] = table_df['entropy'].round(2)
    table_df['gini'] = table_df['gini'].round(3)
    table_df['top1_pct'] = table_df['top1_pct'].round(1).astype(str) + '%'
    table_df['stars'] = table_df['stars'].apply(lambda x: f"{x:,}")
    
    # Rename columns for publication
    table_df.columns = ['Project', 'Classification', 'Confidence', 
                       'H (bits)', 'Gini', 'Top 1%', 'N Contributors', 'Stars']
    
    return table_df

if len(df) > 0:
    table1 = create_classification_table(df)
    
    # Save as CSV and LaTeX
    table1.to_csv(TABLES_DIR / 'table1_classification.csv', index=False)
    table1.to_latex(TABLES_DIR / 'table1_classification.tex', index=False,
                    caption='Project Classification Summary',
                    label='tab:classification')
    
    print(f"\nTable 1: Project Classification Summary\n")
    print(table1.to_string(index=False))
    print(f"\nSaved to {TABLES_DIR}")
else:
    print("No data for table")

## Table 2: Statistical Summary by Category

In [ ]:
def create_statistics_table(df: pd.DataFrame) -> pd.DataFrame:
    """Create statistical summary table by category."""
    stats = df.groupby('category').agg({
        'project': 'count',
        'entropy': ['mean', 'std'],
        'normalized_entropy': ['mean', 'std'],
        'gini': ['mean', 'std'],
        'top1_pct': ['mean', 'std'],
        'contributors': ['mean', 'median'],
        'stars': ['mean', 'median']
    }).round(2)
    
    # Flatten column names
    stats.columns = ['_'.join(col).strip() for col in stats.columns.values]
    stats = stats.reset_index()
    
    return stats

if len(df) > 0 and len(df['category'].unique()) > 1:
    table2 = create_statistics_table(df)
    
    # Save as CSV and LaTeX
    table2.to_csv(TABLES_DIR / 'table2_statistics.csv', index=False)
    table2.to_latex(TABLES_DIR / 'table2_statistics.tex', index=False,
                    caption='Statistical Summary by Governance Category',
                    label='tab:statistics')
    
    print(f"\nTable 2: Statistical Summary by Category\n")
    print(table2.to_string(index=False))
    print(f"\nSaved to {TABLES_DIR}")
else:
    print("Insufficient data for statistical summary")

## Supplementary: Stadium Criteria Visualization

In [ ]:
def create_stadium_criteria_figure(df: pd.DataFrame) -> plt.Figure:
    """Visualize Stadium classification criteria."""
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    thresholds = {
        'top1_pct': (40, 'Top Contributor > 40%'),
        'gini': (0.8, 'Gini > 0.8'),
        'normalized_entropy': (0.6, 'Normalized Entropy < 0.6')
    }
    
    for idx, (metric, (threshold, title)) in enumerate(thresholds.items()):
        ax = axes[idx]
        
        # Color by whether threshold is met
        if metric == 'normalized_entropy':
            colors = ['#E74C3C' if v < threshold else '#3498DB' for v in df[metric]]
        else:
            colors = ['#E74C3C' if v > threshold else '#3498DB' for v in df[metric]]
        
        bars = ax.barh(range(len(df)), df[metric], color=colors, alpha=0.7)
        ax.axvline(x=threshold, color='red', linestyle='--', linewidth=2, label='Threshold')
        
        # Labels
        short_names = [p.split('/')[-1][:12] for p in df['project']]
        ax.set_yticks(range(len(df)))
        ax.set_yticklabels(short_names)
        ax.set_title(title)
        ax.grid(True, alpha=0.3, axis='x')
    
    # Legend
    fig.suptitle('Stadium Classification Criteria', fontsize=14, y=1.02)
    red_patch = mpatches.Patch(color='#E74C3C', label='Meets Stadium Criterion')
    blue_patch = mpatches.Patch(color='#3498DB', label='Does Not Meet')
    fig.legend(handles=[red_patch, blue_patch], loc='upper right', 
               bbox_to_anchor=(0.98, 0.98))
    
    plt.tight_layout()
    return fig

if len(df) > 0:
    fig_supp = create_stadium_criteria_figure(df)
    fig_supp.savefig(FIGURES_DIR / 'fig_supp_stadium_criteria.png')
    fig_supp.savefig(FIGURES_DIR / 'fig_supp_stadium_criteria.pdf')
    print(f"Saved supplementary figure to {FIGURES_DIR}")
    plt.show()
else:
    print("No data available")

## Generate All Outputs Summary

In [ ]:
print("=" * 60)
print("PUBLICATION OUTPUTS GENERATED")
print("=" * 60)

print("\nFigures:")
for f in sorted(FIGURES_DIR.glob('*')):
    print(f"  - {f.name}")

print("\nTables:")
for f in sorted(TABLES_DIR.glob('*')):
    print(f"  - {f.name}")

print("\n" + "=" * 60)
print("Ready for LaTeX/paper integration")
print("=" * 60)

## Export Citation-Ready Summary

In [ ]:
def generate_summary_stats(df: pd.DataFrame, projects: Dict) -> str:
    """Generate summary statistics for paper abstract/results."""
    if len(df) == 0:
        return "No data available for summary"
    
    summary = f"""
SUMMARY STATISTICS FOR PUBLICATION
{'=' * 50}

Dataset:
- Total projects analyzed: {len(df)}
- Total contributors across projects: {df['contributors'].sum():,}
- Combined GitHub stars: {df['stars'].sum():,}

Classification Results:
"""
    
    for cat in df['category'].unique():
        count = len(df[df['category'] == cat])
        pct = count / len(df) * 100
        summary += f"- {cat}: {count} projects ({pct:.1f}%)\n"
    
    summary += f"""
Key Metrics (Mean ± SD):
- Shannon Entropy: {df['entropy'].mean():.2f} ± {df['entropy'].std():.2f} bits
- Gini Coefficient: {df['gini'].mean():.3f} ± {df['gini'].std():.3f}
- Top Contributor %: {df['top1_pct'].mean():.1f}% ± {df['top1_pct'].std():.1f}%

Stadium Projects Characteristics:
"""
    
    stadium_df = df[df['category'] == 'Stadium']
    if len(stadium_df) > 0:
        summary += f"- Average Gini: {stadium_df['gini'].mean():.3f}\n"
        summary += f"- Average Top Contributor: {stadium_df['top1_pct'].mean():.1f}%\n"
        summary += f"- Average Normalized Entropy: {stadium_df['normalized_entropy'].mean():.3f}\n"
    else:
        summary += "- No Stadium projects classified yet\n"
    
    return summary

summary_text = generate_summary_stats(df, projects)
print(summary_text)

# Save summary
with open(TABLES_DIR / 'summary_statistics.txt', 'w') as f:
    f.write(summary_text)
print(f"\nSaved summary to {TABLES_DIR / 'summary_statistics.txt'}")